In [1]:
! pip install gtts pydub SpeechRecognition google-generativeai python-dotenv

In [2]:
import os
import json
import re
import time
import threading
import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
import google.generativeai as genai
from dotenv import load_dotenv
from datetime import datetime
import tempfile

AudioSegment.converter = r"C:\ffmpeg\bin\ffmpeg.exe"
AudioSegment.ffprobe = r"C:\ffmpeg\bin\ffprobe.exe"


d:\Work\company project\Crystal Group\text_speech\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
COMPANY_CONTEXT = """

Company Overview:
- Name: Crystal Group (The Cold Chain Solutions Company)
- Industry: Transportation, Logistics, Supply Chain, and Storage
- Headquarters: Mumbai, Maharashtra, India
- Followers: 43,000+
- Employees: 201–500
- Founded: 1962
- Website: crystalgroup.in

Mission: Safeguard the quality of temperature-sensitive goods with care and dedication, enabling clients to succeed at every stage.
Vision: To be the most trusted leader in cold chain logistics, empowering businesses with seamless, reliable services and peace of mind.

Core Services:
- Cold Storage Warehousing: Advanced warehouses with temperature control (-25°C to +25°C), real-time monitoring, and hygiene protocols.
- Portable Cold Storage: Flexible, pay-per-day portable cold storage containers for rent or sale.
- Refrigerated Transportation: Fleet of over 200 reefer trucks for pan-India delivery of temperature-sensitive goods.
- Supply Chain Solutions: End-to-end logistics, including last-mile delivery, distribution hubs, and project cargo.
- Value-Added Services: Inventory management, real-time tracking, repackaging, sorting, and labeling.
- Dry Containers: Secure, weather-resistant storage for non-temperature-sensitive goods.
- Specialized Storage: Solutions for pharmaceuticals, food, floriculture, chemicals, and more.

Key Features:
- Temperature Range: -25°C to +25°C (some containers up to -30°C to +30°C)
- Container Sizes: 10ft, 20ft, 40ft (with high-cube and multi-temperature options)
- Advanced Technology: Mobile pallet racking, WMS software, remote monitoring, humidity control, and multi-zone temperature settings
- Certifications: ISO 9001-2008 certified
- Locations: Mumbai (HQ), Kolkata, Bhubaneswar, Howrah
- Clientele: Food & beverage, pharmaceuticals, agriculture, floriculture, chemicals, and more.

Frequently Asked Questions (FAQs) and Topics:
- Storage Capabilities: What products can you store? Do you handle both fresh and frozen goods?
- Temperature Control: What temperature ranges do you offer? Can you customize storage temperatures?
- Container Sizes & Capacity: What sizes of containers are available? How much can each container hold?
- Rental & Purchase Options: Can I rent or buy a portable cold storage container? What are the terms?
- Transportation Services: Do you provide refrigerated transport? What is your delivery coverage?
- Inventory Management: How do you track and monitor inventory? Is real-time tracking available?
- Security & Safety: What security measures are in place? Are you compliant with food/pharma safety standards?
- Pricing & Contracts: What are your rates? Do you offer long-term or short-term contracts?
- Value-Added Services: Do you offer labeling, kitting, or other value-added services?
- Power Backup & Reliability: Do you have backup power systems to prevent spoilage?
- Regulatory Compliance: Are you certified for food safety (HACCP, FSMA, GMP, etc.)?
- Site Visits: Can I schedule a visit to your facility?
- Support & Response Time: How quickly can I get a response or support?

Customer Segments:
- Food & Beverage (producers, distributors, retailers)
- Pharmaceuticals & Healthcare (vaccines, medicines, blood)
- Agriculture & Horticulture (farmers, exporters, floriculture)
- Hospitality & Catering (hotels, restaurants)
- E-commerce & Retail (grocery, meal delivery)
- Chemical & Industrial (temperature-sensitive chemicals)

Always greet the customer, clarify their needs, and provide clear, concise, and helpful answers. If you do not know the answer, offer to connect the customer with a human expert.
"""

In [11]:
# Load environment variables
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY") or os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemma-3-27b-it")

# # COMPANY_CONTEXT should be defined here as a string
# COMPANY_CONTEXT = """Paste your full company context here."""

HISTORY_FILE = "freddy_history.json"


In [5]:
def load_history():
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return []

def save_history(history):
    with open(HISTORY_FILE, "w", encoding="utf-8") as f:
        json.dump(history, f)

history = load_history()


In [6]:
def clean_text(text):
    return re.sub(r'[*_`]', '', text)


In [7]:
def speak(text):
    tts = gTTS(text=text, lang='en', slow=False)
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as fp:
        filename = fp.name
    tts.save(filename)
    audio = AudioSegment.from_mp3(filename)
    play(audio)
    os.remove(filename)


In [8]:
def get_freddy_answer(question):
    # Add the current question to history
    history.append({
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "role": "user", 
        "content": question
    })
    # Build the prompt with history for context
    history_prompt = "\n".join(
        [f"User: {item['content']}" if item['role'] == "user" else f"Freddy: {item['content']}" for item in history]
    )
    prompt = (
        f"{COMPANY_CONTEXT}\n"
        f"Conversation so far:\n{history_prompt}\n"
        f"Customer question: {question}\n"
        "As Freddy, always provide a direct, friendly, and concise answer using the company information above. "
        "Maintain a professional tone—do not use emojis or informal symbols. Only ask for clarification if the question is unclear. "
        "Keep your answer under 200 characters. If unsure, offer to connect them to a human expert."
    )
    response = model.generate_content(prompt)
    answer = response.text.strip()
    # Add Freddy's answer to history
    history.append({
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "role": "assistant",
        "content": answer
    })
    save_history(history)
    return answer


In [9]:
recognizer = sr.Recognizer()
greeting = "Hello, I am Freddy. How can I help you today?"
print(greeting)
speak(greeting)

print("\nFreddy Voice Assistant (press Ctrl+C to stop)")
while True:
    try:
        with sr.Microphone() as source:
            print("\nListening...")
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            audio = recognizer.listen(source)
            print("Recognizing...")
            try:
                user_text = recognizer.recognize_google(audio)
                print(f"question: {user_text}")
            except Exception as e:
                print(f"Could not recognize speech: {e}")
                continue

            # Get Freddy's answer
            try:
                answer = get_freddy_answer(user_text)
                answer_clean = clean_text(answer)
                print(f"bot answer: {answer_clean}")
                speak(answer_clean)
            except Exception as e:
                print(f"LLM error: {e}")
            time.sleep(0.5)
    except KeyboardInterrupt:
        print("\nExiting Freddy voice assistant.")
        break


Hello, I am Freddy. How can I help you today?

Freddy Voice Assistant (press Ctrl+C to stop)

Listening...
Recognizing...
question: what does this crystal group do


C:\Users\preet\AppData\Local\Temp\ipykernel_6028\1344146692.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z",
C:\Users\preet\AppData\Local\Temp\ipykernel_6028\1344146692.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z",


bot answer: Hi there! Crystal Group provides comprehensive cold chain logistics: storage, refrigerated transport, and supply chain management for temperature-sensitive goods.

Listening...
Recognizing...
question: headquarter


C:\Users\preet\AppData\Local\Temp\ipykernel_6028\1344146692.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z",
C:\Users\preet\AppData\Local\Temp\ipykernel_6028\1344146692.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z",


bot answer: Hi there! Crystal Group’s headquarters are located in Mumbai, Maharashtra, India.

Listening...
Recognizing...
question: other centres of this company


C:\Users\preet\AppData\Local\Temp\ipykernel_6028\1344146692.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z",
C:\Users\preet\AppData\Local\Temp\ipykernel_6028\1344146692.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z",


bot answer: Hi there! Besides our headquarters in Mumbai, we have centres in Kolkata, Bhubaneswar, and Howrah.

Listening...
Recognizing...
question: you can stop


C:\Users\preet\AppData\Local\Temp\ipykernel_6028\1344146692.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z",
C:\Users\preet\AppData\Local\Temp\ipykernel_6028\1344146692.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z",


bot answer: Okay, understood.

You can stop.

Is there anything else I can help you with today?

Listening...
Recognizing...
Could not recognize speech: 

Listening...
Recognizing...
Could not recognize speech: 

Listening...

Exiting Freddy voice assistant.
